In [1]:
!pip install geopandas matplotlib osmnx

In [264]:
import json
import geopandas
import math
import numpy
import pyproj
import shapely
import osmnx as osm

## Climb profile for min/max non nominal flights (min/max NTOFP)

In [283]:
input_crs = "EPSG:27040"
output_crs = "EPSG:4326"

In [284]:
# Load runway shapefile
runways = geopandas.read_file("./data/runways.geojson")
runways = runways.loc[runways.geometry.length > 1000, ["geometry"]]
runways.crs == input_crs

True

In [334]:
def get_heading(coords) -> float:
    """Calculates the heading of a LineString in degrees (0-360)."""
    dx = coords[-1][0] - coords[0][0]
    dy = coords[-1][1] - coords[0][1]
    return np.arctan2(dy, dx)

In [335]:
# conversion factor used to transform climb rate from feet per minute (ft/min) to meters per second (m/s)
fpm_to_ms_factor = 196.85
# aircraft climbs at a constant ground speed of 75 m/s (≈146 knots).
climb_speed_ms = 75
distance_beyond_runway = 30000
# transformer to revert CRS back to WGS84
transformer = pyproj.Transformer.from_crs(input_crs, output_crs, always_xy=True)


def build_climb_profile(runway: shapely.LineString, climb_rate: int, vertex_density: int = 100):
    runway_length = runway.length
    altitude = numpy.zeros_like(vertex_density)
    distances = numpy.linspace(runway_length, runway_length + distance_beyond_runway, vertex_density)
    # distance / speed = time
    seconds_to_climb = (distances - runway_length) / climb_speed_ms
    # covert climb rate from ft/min to m/s
    altitude = climb_rate / fpm_to_ms_factor * seconds_to_climb
    # Calculate width expansion (15° divergence per ICAO)
    initial_width = 300  # meters (primary surface)
    # 15° total divergence
    width_by_distance = initial_width + 2 * distances * numpy.tan(numpy.radians(7.5))
    widths = numpy.where(distances <= runway_length, initial_width, width_by_distance)
    # Generate left and right boundaries
    coords = numpy.array(runway.coords)
    runway_heading = get_heading(coords)
    x_mid = coords[0][0] + (distances * numpy.cos(runway_heading))
    y_mid = coords[0][1] + (distances * numpy.sin(get_heading(coords)))
    # perpendicular vectors for width expansion
    perp_heading = runway_heading + numpy.pi / 2
    x_left = x_mid + (widths / 2 * numpy.cos(perp_heading))
    y_left = y_mid + (widths / 2 * numpy.sin(perp_heading))
    x_right = x_mid - (widths / 2 * numpy.cos(perp_heading))
    y_right = y_mid - (widths / 2 * numpy.sin(perp_heading))
    # Create 3D polygons
    top_points = list(zip(*transformer.transform(x_left, y_left), altitude))
    bottom_points = list(zip(*transformer.transform(x_right[::-1], y_right[::-1]), altitude[::-1]))
    xyz = top_points + bottom_points
    # Ensure closure
    if xyz[0] != xyz[-1]:
        xyz.append(xyz[0])
    return numpy.array(xyz)

In [336]:
# Regulatory minimum (e.g., engine-out scenario)
MIN_CLIMB_RATE = 1000
min_climb_profiles = [build_climb_profile(runway, MIN_CLIMB_RATE) for runway in runways.geometry]
min_climb_profiles[:3]

[array([[  54.6342778 ,   24.44333498,    0.        ],
        [  54.62841252,   24.4408716 ,   20.52529358],
        [  54.62581022,   24.44226235,   41.05058715],
        [  54.62320788,   24.44365305,   61.57588073],
        [  54.62060548,   24.4450437 ,   82.1011743 ],
        [  54.61800302,   24.4464343 ,  102.62646788],
        [  54.61540051,   24.44782485,  123.15176146],
        [  54.61279795,   24.44921536,  143.67705503],
        [  54.61019533,   24.45060581,  164.20234861],
        [  54.60759265,   24.45199621,  184.72764218],
        [  54.60498993,   24.45338656,  205.25293576],
        [  54.60238714,   24.45477687,  225.77822933],
        [  54.59978431,   24.45616712,  246.30352291],
        [  54.59718141,   24.45755733,  266.82881649],
        [  54.59457847,   24.45894748,  287.35411006],
        [  54.59197547,   24.46033759,  307.87940364],
        [  54.58937241,   24.46172765,  328.40469721],
        [  54.5867693 ,   24.46311765,  348.92999079],
        [ 

In [337]:
# High-performance (e.g., lightly loaded A321)
MAX_CLIMB_RATE = 4000
max_climb_profiles = [build_climb_profile(runway, MAX_CLIMB_RATE) for runway in runways.geometry]
max_climb_profiles[:3]

[array([[  54.6342778 ,   24.44333498,    0.        ],
        [  54.62841252,   24.4408716 ,   82.1011743 ],
        [  54.62581022,   24.44226235,  164.20234861],
        [  54.62320788,   24.44365305,  246.30352291],
        [  54.62060548,   24.4450437 ,  328.40469721],
        [  54.61800302,   24.4464343 ,  410.50587152],
        [  54.61540051,   24.44782485,  492.60704582],
        [  54.61279795,   24.44921536,  574.70822012],
        [  54.61019533,   24.45060581,  656.80939443],
        [  54.60759265,   24.45199621,  738.91056873],
        [  54.60498993,   24.45338656,  821.01174303],
        [  54.60238714,   24.45477687,  903.11291734],
        [  54.59978431,   24.45616712,  985.21409164],
        [  54.59718141,   24.45755733, 1067.31526594],
        [  54.59457847,   24.45894748, 1149.41644025],
        [  54.59197547,   24.46033759, 1231.51761455],
        [  54.58937241,   24.46172765, 1313.61878885],
        [  54.5867693 ,   24.46311765, 1395.71996316],
        [ 

In [347]:
mcp = geopandas.GeoDataFrame(geometry=list(map(shapely.Polygon, max_climb_profiles)))
[a[:,-1] > 731 for a in max_climb_profiles]

[array([False, False, False, False, False, False, False, False, False,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
      

In [345]:
max_climb_profiles[0][:,-1]

array([   0.        ,   82.1011743 ,  164.20234861,  246.30352291,
        328.40469721,  410.50587152,  492.60704582,  574.70822012,
        656.80939443,  738.91056873,  821.01174303,  903.11291734,
        985.21409164, 1067.31526594, 1149.41644025, 1231.51761455,
       1313.61878885, 1395.71996316, 1477.82113746, 1559.92231176,
       1642.02348607, 1724.12466037, 1806.22583467, 1888.32700898,
       1970.42818328, 2052.52935758, 2134.63053189, 2216.73170619,
       2298.83288049, 2380.9340548 , 2463.0352291 , 2545.1364034 ,
       2627.23757771, 2709.33875201, 2791.43992631, 2873.54110062,
       2955.64227492, 3037.74344922, 3119.84462353, 3201.94579783,
       3284.04697213, 3366.14814644, 3448.24932074, 3530.35049504,
       3612.45166935, 3694.55284365, 3776.65401795, 3858.75519226,
       3940.85636656, 4022.95754086, 4105.05871517, 4187.15988947,
       4269.26106377, 4351.36223808, 4433.46341238, 4515.56458668,
       4597.66576099, 4679.76693529, 4761.86810959, 4843.96928

In [291]:
min_climb_profiles.to_file("./data/min_climb_profile.geojson", driver="GeoJSON")
max_climb_profiles.to_file("./data/max_climb_profile.geojson", driver="GeoJSON")

In [299]:
for g in min_climb_profiles.geometry:
    coords = shapely.geometry.mapping(g)["coordinates"]
    print(coords)

(((54.634277796502204, 24.443334980124845, 0.0), (54.62841251598089, 24.440871601803014, 20.525293575839676), (54.62581022388679, 24.442262351076, 41.05058715167935), (54.62320787730593, 24.443653050916534, 61.57588072751902), (54.620605476238424, 24.445043701316134, 82.1011743033587), (54.618003020684334, 24.446434302266326, 102.62646787919839), (54.61540051064379, 24.447824853758632, 123.15176145503804), (54.61279794611687, 24.449215355784546, 143.67705503087774), (54.61019532710369, 24.450605808335602, 164.2023486067174), (54.60759265360434, 24.451996211403312, 184.72764218255708), (54.60498992561892, 24.453386564979198, 205.25293575839677), (54.60238714314753, 24.454776869054772, 225.77822933423644), (54.59978430619029, 24.456167123621555, 246.30352291007608), (54.59718141474728, 24.457557328671072, 266.82881648591575), (54.5945784688186, 24.45894748419483, 287.3541100617555), (54.59197546840438, 24.460337590184366, 307.8794036375952), (54.58937241350469, 24.46172764663117, 328.404